In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as np
import re
import math

# 데이터 다운로드
* ratings.csv: https://drive.google.com/file/d/11etcUuNG8cG0sZY0BOQDtwZuK_4e5GIe/view?usp=sharing
* movies.csv: https://drive.google.com/file/d/1ZIahWxaeUXBg6bqF9z_rA5DkiJxYuFrP/view?usp=sharing

In [ ]:
rating_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/RS/ratings.csv')
movie_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/RS/movies.csv')

In [ ]:
rating_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movie_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
rating_data.drop('timestamp', axis=1, inplace=True)
rating_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
movie_data.drop('genres', axis=1, inplace=True)
movie_data.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [ ]:
# movieId를 기준으로 두 데이터프레임을 합치기
user_movie_data = pd.merge(rating_data, movie_data, on='movieId')
user_movie_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
# 인덱스: 유저 아이디, 컬럼: 영화 제목
user_movie_pivot = user_movie_data.pivot_table('rating', index='userId', columns='title')
user_movie_id_pivot = user_movie_data.pivot_table('rating', index='userId', columns='movieId')
user_movie_id_pivot

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.5,NaN,4.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,4.5,NaN,NaN,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,2.0,NaN,3.5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.5,3.5,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# NaN에 컬럼별 평균 채우기 -> 행렬 모든 값에서 각 행 평균 빼주기(정규화)
# 사용자마다 영화를 매기는 기준이 다르기 때문에(관대하거나 엄격하거나) 정규화 필요
def R_filled_in(user_movie_id_pivot):
  for col in range(len(user_movie_id_pivot.columns)):
    col_update = []
    col_num = [i for i in user_movie_id_pivot.iloc[:, col] if math.isnan(i)==False] # NaN 값 제외
    col_mean = sum(col_num) / len(col_num)

    # 기존값은 그대로, NaN 값은 컬럼 평균으로 대체
    col_update = [i if math.isnan(i)==False else col_mean for i in user_movie_id_pivot.iloc[:, col]]

    user_movie_id_pivot.iloc[:, col] = col_update

  return user_movie_id_pivot

In [ ]:
user_movie_R_filled = R_filled_in(user_movie_id_pivot)
user_movie_R_filled

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.00000,3.431818,4.000000,2.357143,3.071429,4.000000,3.185185,2.875,3.125,3.496212,3.671429,2.421053,3.125,3.833333,3.0,3.926829,3.776119,3.7,2.727273,2.5,3.494382,3.222222,3.125,3.125,3.625,3.5,3.333333,4.227273,4.013158,3.0,3.184211,3.983051,3.652344,3.835821,2.125,3.293269,4.25,3.7,3.0,3.1875,...,4.75,3.0,3.0,2.75,3.0,3.25,3.0,3.875,3.9,5.0,4.5,3.666667,3.5,4.5,4.0,4.5,2.5,3.0,3.75,2.5,1.0,2.5,3.5,1.5,4.0,1.0,1.5,1.0,1.0,4.5,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,3.671429,2.421053,3.125,3.833333,3.0,3.926829,3.776119,3.7,2.727273,2.5,3.494382,3.222222,3.125,3.125,3.625,3.5,3.333333,4.227273,4.013158,3.0,3.184211,3.983051,3.652344,3.835821,2.125,3.293269,4.25,3.7,3.0,3.1875,...,4.75,3.0,3.0,2.75,3.0,3.25,3.0,3.875,3.9,5.0,4.5,3.666667,3.5,4.5,4.0,4.5,2.5,3.0,3.75,2.5,1.0,2.5,3.5,1.5,4.0,1.0,1.5,1.0,1.0,4.5,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,3.671429,2.421053,3.125,3.833333,3.0,3.926829,3.776119,3.7,2.727273,2.5,3.494382,3.222222,3.125,3.125,3.625,3.5,3.333333,4.227273,4.013158,3.0,0.500000,3.983051,3.652344,3.835821,2.125,3.293269,4.25,3.7,3.0,3.1875,...,4.75,3.0,3.0,2.75,3.0,3.25,3.0,3.875,3.9,5.0,4.5,3.666667,3.5,4.5,4.0,4.5,2.5,3.0,3.75,2.5,1.0,2.5,3.5,1.5,4.0,1.0,1.5,1.0,1.0,4.5,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,3.671429,2.421053,3.125,3.833333,3.0,3.926829,3.776119,3.7,2.727273,2.5,3.000000,3.222222,3.125,3.125,3.625,3.5,3.333333,4.227273,4.013158,3.0,3.184211,2.000000,3.652344,3.835821,2.125,3.293269,4.25,3.7,3.0,3.1875,...,4.75,3.0,3.0,2.75,3.0,3.25,3.0,3.875,3.9,5.0,4.5,3.666667,3.5,4.5,4.0,4.5,2.5,3.0,3.75,2.5,1.0,2.5,3.5,1.5,4.0,1.0,1.5,1.0,1.0,4.5,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
5,4.00000,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,3.671429,2.421053,3.125,3.833333,3.0,3.926829,3.776119,3.7,2.727273,2.5,4.000000,3.222222,3.125,3.125,3.625,3.5,3.333333,4.227273,4.013158,3.0,3.184211,3.983051,4.000000,4.000000,2.125,3.000000,4.25,3.7,3.0,3.1875,...,4.75,3.0,3.0,2.75,3.0,3.25,3.0,3.875,3.9,5.0,4.5,3.666667,3.5,4.5,4.0,4.5,2.5,3.0,3.75,2.5,1.0,2.5,3.5,1.5,4.0,1.0,1.5,1.0,1.0,4.5,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.50000,3.431818,3.259615,2.357143,3.071429,3.946078,2.500000,2.875,3.125,3.496212,2.500000,2.421053,3.125,3.833333,3.5,3.926829,4.000000,4.0,2.000000,2.5,3.494382,3.222222,3.125,3.125,3.625,3.5,3.333333,3.500000,4.500000,3.0,3.184211,4.000000,3.652344,3.500000,2.125,3.293269,4.25,3.7,3.0,3.1875,...,4.75,3.0,3.0,2.75,3.0,3.25,3.0,3.875,3.9,5.0,4.5,3.666667,3.5,4.5,4.0,4.5,2.5,3.0,3.75,2.5,1.0,2.5,3.5,1.5,4.0,1.0,1.5,1.0,1.0,4.5,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
607,4.00000,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,3.000000,2.421053,3.125,3.833333,3.0,3.926829,3.776119,3.7,2.727273,2.5,3.494382,3.222222,3.125,3.125,3.000,3.5,3.333333,4.227273,4.013158,3.0,3.184211,3.983051,3.000000,4.000000,2.125,3.293269,4.25,3.7,3.0,3.1875,...,4.75,3.0,3.0,2.75,3.0,3.25,3.0

In [ ]:
def R_norm(user_movie_R_filled):
  for row in range(len(user_movie_R_filled)):
    row_update = []
    row_mean = sum(user_movie_R_filled.iloc[row, :]) / len(user_movie_R_filled.iloc[row, :])
    row_update = [i - row_mean for i in user_movie_R_filled.iloc[row, :]]

    user_movie_R_filled.iloc[row, :] = row_update

  return user_movie_R_filled

user_movie_R_norm = R_norm(user_movie_R_filled)
user_movie_R_norm

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.718247,0.150065,0.718247,-0.924611,-0.210325,0.718247,-0.096568,-0.406753,-0.156753,0.214459,0.389675,-0.860701,-0.156753,0.551580,-0.281753,0.645076,0.494366,0.418247,-0.554481,-0.781753,0.212629,-0.059531,-0.156753,-0.156753,0.343247,0.218247,0.051580,0.945519,0.731405,-0.281753,-0.097543,0.701297,0.370590,0.554068,-1.156753,0.011516,0.968247,0.418247,-0.281753,-0.094253,...,1.468247,-0.281753,-0.281753,-0.531753,-0.281753,-0.031753,-0.281753,0.593247,0.618247,1.718247,1.218247,0.384913,0.218247,1.218247,0.718247,1.218247,-0.781753,-0.281753,0.468247,-0.781753,-2.281753,-0.781753,0.218247,-1.781753,0.718247,-2.281753,-1.781753,-2.281753,-2.281753,1.218247,0.218247,-0.281753,0.718247,0.718247,0.218247,0.718247,0.218247,0.218247,0.218247,0.718247
2,0.658467,0.169355,-0.002848,-0.905321,-0.191035,0.683615,-0.077278,-0.387464,-0.137464,0.233748,0.408965,-0.841411,-0.137464,0.570870,-0.262464,0.664366,0.513656,0.437536,-0.535191,-0.762464,0.231918,-0.040241,-0.137464,-0.137464,0.362536,0.237536,0.070870,0.964809,0.750694,-0.262464,-0.078253,0.720587,0.389880,0.573357,-1.137464,0.030806,0.987536,0.437536,-0.262464,-0.074964,...,1.487536,-0.262464,-0.262464,-0.512464,-0.262464,-0.012464,-0.262464,0.612536,0.637536,1.737536,1.237536,0.404203,0.237536,1.237536,0.737536,1.237536,-0.762464,-0.262464,0.487536,-0.762464,-2.262464,-0.762464,0.237536,-1.762464,0.737536,-2.262464,-1.762464,-2.262464,-2.262464,1.237536,0.237536,-0.262464,0.737536,0.737536,0.237536,0.737536,0.237536,0.237536,0.237536,0.737536
3,0.663091,0.173979,0.001776,-0.900697,-0.186411,0.688239,-0.072654,-0.382839,-0.132839,0.238373,0.413589,-0.836787,-0.132839,0.575494,-0.257839,0.668990,0.518280,0.442161,-0.530567,-0.757839,0.236543,-0.035617,-0.132839,-0.132839,0.367161,0.242161,0.075494,0.969433,0.755318,-0.257839,-2.757839,0.725211,0.394504,0.577981,-1.132839,0.035430,0.992161,0.442161,-0.257839,-0.070339,...,1.492161,-0.257839,-0.257839,-0.507839,-0.257839,-0.007839,-0.257839,0.617161,0.642161,1.742161,1.242161,0.408827,0.242161,1.242161,0.742161,1.242161,-0.757839,-0.257839,0.492161,-0.757839,-2.257839,-0.757839,0.242161,-1.757839,0.742161,-2.257839,-1.757839,-2.257839,-2.257839,1.242161,0.242161,-0.257839,0.742161,0.742161,0.242161,0.742161,0.242161,0.242161,0.242161,0.742161
4,0.662843,0.173731,0.001528,-0.900944,-0.186658,0.687991,-0.072902,-0.383087,-0.133087,0.238125,0.413342,-0.837034,-0.133087,0.575246,-0.258087,0.668742,0.518032,0.441913,-0.530814,-0.758087,-0.258087,-0.035865,-0.133087,-0.133087,0.366913,0.241913,0.075246,0.969186,0.755071,-0.258087,-0.073876,-1.258087,0.394257,0.577734,-1.133087,0.035182,0.991913,0.441913,-0.258087,-0.070587,...,1.491913,-0.258087,-0.258087,-0.508087,-0.258087,-0.008087,-0.258087,0.616913,0.641913,1.741913,1.241913,0.408580,0.241913,1.241913,0.741913,1.241913,-0.758087,-0.258087,0.491913,-0.758087,-2.258087,-0.758087,0.241913,-1.758087,0.741913,-2.258087,-1.758087,-2.258087,-2.258087,1.241913,0.241913,-0.258087,0.741913,0.741913,0.241913,0.741913,0.241913,0.241913,0.241913,0.741913
5,0.737895,0.169713,-0.002490,-0.904963,-0.190677,0.683973,-0.076920,-0.387105,-0.137105,0.234107,0.409323,-0.841053,-0.137105,0.571228,-0.262105,0.664724,0.514014,0.437895,-0.534833,-0.762105,0.737895,-0.039883,-0.137105,-0.137105,0.362895,0.237895,0.071228,0.965167,0.751053,-0.262105,-0.077895,0.720945,0.737895,0.737895,-1.137105,-0.262105,0.987895,0.437895,-0.262105,-0.074605,...,1.487895,-0.262105,-0.262105,-0.512105,-0.262105,-0.012105,-0.262105,0.61289

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Sigma 차원을 12로, 즉 (12x12) 특잇값 행렬
svd = TruncatedSVD(n_components=12, n_iter=5) # n_iter: 랜덤 SVD 계산기의 반복횟수
svd.fit(np.array(user_movie_R_norm))

U = svd.fit_transform(np.array(user_movie_R_norm))
Simga = svd.explained_variance_ratio_
VT = svd.components_

In [ ]:
# U=(mxk) / S=(kxk) / VT=(kxn) => 원본 행렬=(mxn)
print(U.shape, Simga.shape, VT.shape)

(610, 12) (12,) (12, 9724)


In [ ]:
Simga_sqrt = []
for i in range(len(Simga)):
  tmp = []
  tmp = [math.sqrt(s) for s in np.diag(Simga)[i]]
  Simga_sqrt.append(tmp)

Simga_sqrt = np.array(Simga_sqrt)

# m 사용자에 대한 특징(=취향) 크기가 12인 행렬
U_S_sqrt = pd.DataFrame(np.matmul(U, Simga_sqrt))
U_S_sqrt

,0,1,2,3,4,5,6,7,8,9,10,11
0,2.296607,-0.597112,0.356846,0.305494,0.021049,-0.138014,0.027932,-0.031074,-0.131169,-0.111277,0.053669,0.036422
1,2.282947,0.014777,0.005312,-0.019848,0.004198,-0.000995,0.001877,-0.010537,-0.005748,0.000398,-0.000929,0.011618
2,2.278928,0.130981,-0.012797,-0.099913,0.056503,0.013143,0.105617,-0.056624,-0.136050,0.093788,-0.039528,0.031840
3,2.279885,0.278405,0.058840,-0.162247,-0.115924,-0.146880,-0.038791,0.081876,-0.079504,-0.008264,-0.030315,0.159065
4,2.282980,0.061134,-0.015836,0.013342,-0.018232,-0.003818,-0.023524,-0.007940,-0.016422,0.008079,0.005720,0.007737
...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.279828,0.176945,-0.158925,0.225665,-0.185503,-0.002962,-0.093737,-0.108312,0.234331,-0.027646,0.164161,-0.072206
606,2.279401,-0.166845,0.225518,-0.122378,0.003444,-0.011316,0.025640,0.012428,0.165156,0.016259,-0.151766,-0.101083
607,2.282897,0.465515,-1.436545,-0.395784,0.457801,0.951530,0.432840,0.086917,-0.124344,-0.208669,-0.440686,-1.265405
608,2.281025,0.084216,-0.042265,-0.019464,-0.023455,0.019138,0.006897,0.003929,-0.039477,0.009255,-0.005641,0.005180


In [ ]:
# n 아이템에 대한 특징(영화 성격: 로맨스, 액션, 코미디 등) 크기가 12인 행렬
VT_S_sqrt = pd.DataFrame(np.matmul(Simga_sqrt, VT))
VT_S_sqrt

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9684,9685,9686,9687,9688,9689,9690,9691,9692,9693,9694,9695,9696,9697,9698,9699,9700,9701,9702,9703,9704,9705,9706,9707,9708,9709,9710,9711,9712,9713,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,0.000204,0.000053,-8.389717e-07,-2.809332e-04,-0.000059,0.000212,-0.000024,-0.000120,-0.000043,0.000073,0.000127,-0.000261,-0.000043,0.000177,-0.000081,0.000206,0.000159,0.000136,-0.000166,-0.000237,0.000072,-0.000013,-0.000043,-0.000043,0.000113,0.000074,0.000022,0.000299,0.000233,-0.000081,-0.000024,0.000224,0.000121,0.000178,-0.000353,0.000010,0.000306,0.000136,-0.000081,-0.000023,...,0.000462,-8.144766e-05,-8.144766e-05,-0.000159,-8.144766e-05,-0.000004,-0.000081,0.000190,0.000198,0.000539,0.000384,0.000125,0.000074,0.000384,0.000229,0.000384,-0.000237,-8.144766e-05,0.000151,-0.000237,-0.000702,-0.000237,0.000074,-0.000547,0.000229,-0.000702,-0.000547,-0.000702,-0.000702,0.000384,0.000074,-8.144766e-05,0.000229,0.000229,0.000074,0.000229,0.000074,0.000074,0.000074,0.000229
1,-0.013827,-0.013150,-7.332552e-03,6.736101e-05,-0.002255,-0.002670,-0.001161,0.000452,-0.001785,-0.006520,-0.003901,-0.002356,0.000339,0.001104,-0.000910,-0.000274,-0.003274,-0.001617,-0.007852,-0.001140,-0.002026,-0.001715,-0.001606,-0.002679,-0.000407,-0.000276,0.001066,0.000716,-0.000808,0.001274,-0.002806,-0.007549,-0.004524,0.000602,0.000824,-0.004071,0.000939,-0.000049,0.001250,0.000560,...,0.001037,7.894941e-04,7.894941e-04,0.000817,7.894941e-04,0.000826,0.000673,0.000414,0.000938,0.001135,0.001049,0.000506,0.000876,0.001049,0.000962,0.001049,0.000703,7.894941e-04,0.000908,0.000703,0.000444,0.000703,0.000876,0.000530,0.000962,0.000444,0.000530,0.000444,0.000444,0.001049,0.000876,7.894941e-04,0.000962,0.000962,0.000876,0.000962,0.000876,0.000876,0.000876,0.000962
2,0.003058,-0.000009,7.458910e-03,6.738775e-04,0.004657,0.003449,0.003023,0.000613,0.001834,0.005931,0.002916,0.002250,0.000694,0.000244,0.002486,0.001021,-0.000805,0.002251,0.002637,0.000738,-0.000134,0.004076,0.002436,0.004797,0.001042,0.000668,0.002153,-0.000162,0.000864,-0.000675,0.003320,-0.002320,0.002963,0.000168,0.001327,0.004328,0.000054,0.000559,0.000724,0.000181,...,-0.000034,-1.657846e-05,-1.657846e-05,0.000014,-1.657846e-05,-0.000069,-0.000086,-0.000874,0.000036,0.000004,-0.000001,-0.000049,-0.000011,-0.000001,-0.000006,-0.000001,-0.000022,-1.657846e-05,-0.000015,-0.000022,-0.000037,-0.000022,-0.000011,-0.000032,-0.000006,-0.000037,-0.000032,-0.000037,-0.000037,-0.000001,-0.000011,-1.657846e-05,-0.000006,-0.000006,-0.000011,-0.000006,-0.000011,-0.000011,-0.000011,-0.000006
3,0.001230,-0.004799,5.905185e-04,7.974932e-07,-0.002807,-0.000387,-0.000828,-0.000278,-0.002639,-0.004872,-0.002745,-0.002075,0.000075,0.000488,-0.004356,0.000301,0.002632,0.000127,0.001304,-0.000612,0.002018,-0.002375,-0.004990,-0.001320,0.002179,-0.000358,-0.002517,0.000356,0.000149,0.000498,-0.002794,0.002855,0.006716,-0.000278,0.000849,-0.001830,0.000026,-0.001168,-0.001690,-0.000426,...,-0.000075,2.176495e-04,2.176495e-04,0.000225,2.176495e-04,0.000157,0.000210,0.000211,-0.000777,-0.000100,-0.000021,0.000131,0.000138,-0.000021,0.000059,-0.000021,0.000297,2.176495e-04,0.000106,0.000297,0.000535,0.000297,0.000138,0.000456,0.000059,0.000535,0.000456,0.000535,0.000535,-0.000021,0.000138,2.176495e-04,0.000059,0.000059,0.000138,0.000059,0.000138,0.000138,0.000138,0.000059
4,-0.002695,-0.001468,-6.336537e-03,3.055606e-04,0.000170,0.005710,-0.002712,0.000148,-0.003533,0.009168,-0.001487,-0.000789,0.000368,-0.000020,-0.003122,-0.004549,0.000678,-0.002565,0.003801,-0.002495,0.002358,0.002895,-0.005221,-0.001074,-0.000282,-0.003151,0.000672,0.000468,-0.001109,-0.000347,-0.002465,-0.001193,-0.002291,0.000432,-0.000843,-0.001315,0.000273,-0.003245,0.000789,-0.000234,...,0.000318,1.854578e-04,1.854578e-04,0.000191,1.854578e-04,0.000208,0.000224,-0.000006,0.000747,0.000312,0.000280,0.000489,0.000217,0.00

In [ ]:
def add_user_mean(user_movie_R_filled):
  row_mean_list = []

  for row in range(len(user_movie_R_filled)):
      row_update = []
      row_mean = sum(user_movie_R_filled.iloc[row, :]) / len(user_movie_R_filled.iloc[row, :])
      base = pd.DataFrame(np.matmul(np.matmul(U, np.diag(Simga)), VT), columns=user_movie_id_pivot.columns) # SVT 행렬곱

      row_update = [i + row_mean for i in base.iloc[row, :]] # 이전 단계에서 뺏던 사용자별 평균 더하기
      user_movie_R_filled.iloc[row, :] = row_update

  return user_movie_R_filled

user_movie_R_red = add_user_mean(user_movie_R_filled)
user_movie_R_red

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.010979,0.008015,0.007178,-0.000670,0.002509,0.003843,0.001663,-0.000345,0.000435,0.004740,0.003589,0.000904,-0.000008,0.000270,-0.000435,0.000692,0.001738,0.001915,0.006343,0.000056,0.000417,0.001983,-0.000462,0.002502,0.001241,0.000204,-0.000241,-0.000055,0.002487,-0.001533,0.001672,0.005149,0.005328,-0.000732,-0.000010,0.002731,0.000183,-0.000085,-0.001249,-0.000568,...,0.000427,-0.000595,-0.000595,-0.000773,-0.000595,-4.715415e-04,-0.000562,-0.000180,-0.000340,0.000554,0.000267,0.000042,-0.000308,0.000267,-0.000021,0.000267,-0.000882,-0.000595,-0.000158,-0.000882,-0.001744,-0.000882,-0.000308,-0.001456,-0.000021,-0.001744,-0.001456,-0.001744,-0.001744,0.000267,-0.000308,-0.000595,-0.000021,-0.000021,-0.000308,-0.000021,-0.000308,-0.000308,-0.000308,-0.000021
2,0.000309,0.000173,-0.000050,-0.000661,-0.000051,0.000536,-0.000007,-0.000250,-0.000083,0.000197,0.000296,-0.000528,-0.000090,0.000425,-0.000110,0.000391,0.000172,0.000293,-0.000311,-0.000562,0.000068,0.000023,-0.000039,-0.000112,0.000189,0.000161,0.000158,0.000694,0.000531,-0.000178,-0.000057,0.000384,0.000071,0.000380,-0.000798,0.000018,0.000712,0.000315,-0.000121,-0.000044,...,0.001070,-0.000180,-0.000180,-0.000357,-0.000180,-1.669755e-07,-0.000181,0.000417,0.000490,0.001250,0.000892,0.000301,0.000178,0.000892,0.000535,0.000892,-0.000537,-0.000180,0.000356,-0.000537,-0.001609,-0.000537,0.000178,-0.001252,0.000535,-0.001609,-0.001252,-0.001609,-0.001609,0.000892,0.000178,-0.000180,0.000535,0.000535,0.000178,0.000535,0.000178,0.000178,0.000178,0.000535
3,-0.001737,-0.000851,-0.001268,-0.000815,-0.000441,0.001050,-0.000079,-0.000114,-0.000192,0.000173,0.000716,-0.000217,-0.000117,0.000619,-0.000351,-0.000572,-0.001932,-0.000333,0.000236,-0.000715,-0.000392,0.000291,-0.000524,-0.000879,-0.000700,-0.000049,0.000249,0.001108,0.000034,0.000152,-0.000755,-0.000602,-0.001059,-0.000335,-0.000622,-0.000586,0.000828,0.000271,-0.000090,-0.000011,...,0.001216,-0.000100,-0.000100,-0.000273,-0.000100,1.042681e-04,-0.000100,0.000236,0.000642,0.001413,0.001035,0.000477,0.000278,0.001035,0.000657,0.001035,-0.000478,-0.000100,0.000466,-0.000478,-0.001613,-0.000478,0.000278,-0.001235,0.000657,-0.001613,-0.001235,-0.001613,-0.001613,0.001035,0.000278,-0.000100,0.000657,0.000657,0.000278,0.000657,0.000278,0.000278,0.000278,0.000657
4,-0.002427,-0.001148,-0.000796,-0.000817,-0.000197,-0.001445,-0.000022,-0.000017,0.000134,-0.001126,0.000874,-0.000709,0.000094,0.000765,0.000357,-0.000679,-0.001152,-0.000571,-0.003623,-0.000790,-0.001275,-0.000106,0.000228,0.000032,-0.000868,0.000220,0.000833,0.000632,0.000634,-0.000002,0.000414,-0.002529,-0.002555,-0.000279,-0.000345,0.001250,0.000949,0.000621,0.000129,0.000173,...,0.001328,-0.000029,-0.000029,-0.000185,-0.000029,1.517499e-04,-0.000083,0.000345,0.000622,0.001554,0.001159,0.000338,0.000367,0.001159,0.000763,0.001159,-0.000425,-0.000029,0.000558,-0.000425,-0.001612,-0.000425,0.000367,-0.001216,0.000763,-0.001612,-0.001216,-0.001612,-0.001612,0.001159,0.000367,-0.000029,0.000763,0.000763,0.000367,0.000763,0.000367,0.000367,0.000367,0.000763
5,-0.000154,-0.000531,-0.000459,-0.000686,-0.000353,0.000267,-0.000145,-0.000262,-0.000229,-0.000359,-0.000021,-0.000764,-0.000091,0.000489,-0.000269,0.000548,0.000244,0.000215,-0.000822,-0.000571,-0.000067,-0.000282,-0.000193,-0.000374,0.000256,0.000170,0.000102,0.000740,0.000449,-0.000049,-0.000280,-0.000016,-0.000116,0.000441,-0.000682,-0.000228,0.000751,0.000325,-0.000119,-0.000031,...,0.001110,-0.000138,-0.000138,-0.

In [ ]:
def recommend_movies(pred_df, userId, movies_df, original_ratings_df, num_recomm):
  user_row_number= userId - 1 # userId starts at 1, not 0

  # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 평점이 높은 순으로 정렬
  # Caution: pred_df.iloc[329] = userId '330'
  sorted_user_predictions = pred_df.iloc[user_row_number].sort_values(ascending=False)

  # 원본 데이터에서 uderId에 해당하는 데이터 추출
  user_data = original_ratings_df[original_ratings_df['userId'] == userId]

  # 위에서 뽑은 user_data와 원본 영화 데이터를 합침 => movies a user already watched
  user_history = user_data.merge(movies_df, how='left', left_on='movieId', right_on='movieId').sort_values(['rating'], ascending=False)

  # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터 추출
  recommendations = movies_df[~movies_df['movieId'].isin(user_history['movieId'])]

  # 사용자 평점이 높은 순으로 정렬된 데이터(예측)와 위의 recommendations를 합침
  recommendations_merge = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left', left_on='movieId', right_on='movieId')

  # 컬럼 이름 바꾸고 정렬 
  # Caution: the original name of column is userId, not user_row_number
  final_recommendation = recommendations_merge.rename(columns = {userId: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recomm, :]

  return user_history, final_recommendation

In [ ]:
already_rated, predictions = recommend_movies(user_movie_R_red, 330, movie_data, rating_data, 10)
already_rated.head()

,userId,movieId,rating,timestamp,title,genres
246,330,79702,5.0,1285903928,Scott Pilgrim vs. the World (2010),Action|Comedy|Fantasy|Musical|Romance
155,330,2324,5.0,1285904320,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War
208,330,4886,5.0,1285905314,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy
204,330,4226,5.0,1285904361,Memento (2000),Mystery|Thriller
202,330,4022,5.0,1285905457,Cast Away (2000),Drama


In [ ]:
predictions.head()

,movieId,title,genres,Predictions
6074,48385,Borat: Cultural Learnings of America for Make ...,Comedy,0.002838
1166,1748,Dark City (1998),Adventure|Film-Noir|Sci-Fi|Thriller,0.002712
818,1221,"Godfather: Part II, The (1974)",Crime|Drama,0.002304
6796,68954,Up (2009),Adventure|Animation|Children|Drama,0.002214
2077,3000,Princess Mononoke (Mononoke-hime) (1997),Action|Adventure|Animation|Drama|Fantasy,0.002138
